# 🔍 PaddleOCR-VL Demo on AMD GPU

This notebook demonstrates how to use **PaddleOCR-VL** (Vision-Language OCR) model running on AMD GPU with vLLM backend.

## Features
- **Layout Detection**: Detect document structure (text, images, tables, titles)
- **VL Recognition**: Vision-Language model for accurate text recognition
- **vLLM Backend**: High-performance inference on AMD GPU


## 1. Check vLLM Server Status

First, let's verify that the vLLM server is running and ready.


In [ ]:
import requests
import json

# Check vLLM server status
try:
    response = requests.get("http://localhost:8118/v1/models")
    models = response.json()
    print("✅ vLLM Server is running!")
    print(f"Available models: {[m['id'] for m in models['data']]}")
except Exception as e:
    print(f"❌ vLLM Server not ready: {e}")
    print("Please wait a moment and retry...")


## 2. Load PaddleOCR-VL Pipeline

Load the PaddleOCR-VL pipeline with vLLM backend configuration.


In [ ]:
import os
os.chdir("/opt/PaddleX")

# Set environment variable to skip model source check
os.environ["PADDLE_PDX_DISABLE_MODEL_SOURCE_CHECK"] = "True"

from paddlex import create_pipeline

# Create the PaddleOCR-VL pipeline with vLLM backend
pipeline = create_pipeline(pipeline="PaddleOCR-VL-vllm.yaml")
print("✅ Pipeline loaded successfully!")


## 3. Display Test Image

Let's view the demo image before running OCR.


In [ ]:
from IPython.display import Image, display

# Display the test image
test_image = "/opt/PaddleX/test/paddleocr_vl_demo.png"
print(f"Test image: {test_image}")
display(Image(filename=test_image, width=600))


## 4. Run OCR Inference

Execute the PaddleOCR-VL pipeline on the test image.


In [ ]:
%%time

# Run inference
result = pipeline.predict(test_image)

print("✅ Inference completed!")


## 5. View Results

Display the extracted text content from the document.


In [ ]:
# Process and display results
for res in result:
    # Get parsing results - res is a PaddleOCRVLResult (dict-like)
    parsing_list = res.get('parsing_res_list', [])
    
    print("=" * 60)
    print("📄 EXTRACTED CONTENT")
    print("=" * 60)
    
    for i, block in enumerate(parsing_list):
        # block is a PaddleOCRVLBlock object, access attributes directly
        label = getattr(block, 'label', 'unknown')
        content = getattr(block, 'content', '')
        
        if content:  # Only show blocks with content
            label_emoji = {
                'doc_title': '📌',
                'paragraph_title': '📍',
                'text': '📝',
                'image': '🖼️',
                'table': '📊',
                'vision_footnote': '📎'
            }.get(label, '•')
            
            print(f"\n{label_emoji} [{label.upper()}]")
            print("-" * 40)
            print(content[:500] + ('...' if len(content) > 500 else ''))


---

## 🎉 Congratulations!

You have successfully run PaddleOCR-VL on AMD GPU with vLLM backend.

### Resources
- [PaddleX Documentation](https://paddlepaddle.github.io/PaddleX/)
- [PaddleOCR-VL Tutorial](https://github.com/PaddlePaddle/PaddleX)
- [AMD ROCm](https://rocm.docs.amd.com/)
